# Procesamiento del Lenguaje Natural 
El Procesamiento del Lenguaje Natural (PNL) es una disciplina de la informática que se ocupa de la comunicación entre los lenguajes naturales (humanos) y los lenguajes informáticos. Un ejemplo común de PNL es algo como el corrector ortográfico o el autocompletado. Esencialmente, la PNL es el campo que se centra en cómo los ordenadores pueden entender y/o procesar los lenguajes naturales/humanos. 

### Redes Neuronales Recurrentes

En este tutorial introduciremos un nuevo tipo de red neuronal que es mucho más capaz de procesar datos secuenciales como texto o caracteres, llamada **red neuronal recurrente** (RNN para abreviar). 

Aprenderemos a utilizar una red neuronal recurrente para hacer lo siguiente:
- Análisis de Sentimientos
- Generación de caracteres 

Las RNN son complejas y se presentan en muchas formas diferentes, por lo que en este tutorial nos centraremos en cómo funcionan y en el tipo de problemas para los que son más adecuadas.

## Datos de la secuencia
En los tutoriales anteriores nos centramos en datos que podíamos representar como un punto de datos estático donde la noción de tiempo o paso era irrelevante. Tomemos por ejemplo nuestros datos de imagen, era simplemente un tensor de forma (ancho, alto, canales). Esos datos no cambian ni les importa la noción de tiempo. 

En este tutorial veremos las secuencias de texto y aprenderemos cómo podemos codificarlas de forma significativa. A diferencia de las imágenes, los datos secuenciales, como las largas cadenas de texto, los patrones climáticos, los vídeos y, en realidad, cualquier cosa en la que la noción de paso o tiempo sea relevante, necesita ser procesada y manejada de una manera especial. 

¿Pero qué quiero decir con secuencias y por qué los datos de texto son una secuencia? Bueno, esa es una buena pregunta. Dado que los datos textuales contienen muchas palabras que se suceden en un orden muy específico y significativo, tenemos que ser capaces de seguir la pista de cada palabra y de cuándo aparece en los datos. Codificar simplemente, por ejemplo, un párrafo entero de texto en un punto de datos no nos daría una imagen muy significativa de los datos y sería muy difícil hacer algo con ellos. Por eso tratamos el texto como una secuencia y procesamos una palabra cada vez. Seguiremos la pista de dónde aparece cada una de esas palabras y utilizaremos esa información para intentar comprender el significado de los trozos de texto.

## Codificación del texto
Como sabemos, los modelos de aprendizaje automático y las redes neuronales no aceptan datos de texto en bruto como entrada. Esto significa que debemos codificar de alguna manera nuestros datos textuales en valores numéricos que nuestros modelos puedan entender. Hay muchas formas de hacerlo y a continuación veremos algunos ejemplos. 

Antes de entrar en los diferentes métodos de codificación/preprocesamiento, entendamos la información que podemos obtener de los datos textuales observando las siguientes dos críticas de películas.

Pensé que la película iba a ser mala, pero en realidad fue increíble.

Pensé que la película iba a ser increíble, pero en realidad era mala".

Aunque estas dos frases son muy parecidas, sabemos que tienen significados muy diferentes. Esto se debe a la **ordenación** de las palabras, una propiedad muy importante de los datos textuales.

Ahora, tenlo en cuenta mientras consideramos algunas formas diferentes de codificar nuestros datos textuales.

### Bolsa de palabras
La primera y más sencilla forma de codificar nuestros datos es utilizar algo llamado **bolsa de palabras**. Se trata de una técnica bastante sencilla en la que cada palabra de una frase se codifica con un número entero y se arroja a una colección que no mantiene el orden de las palabras pero sí la frecuencia. Echa un vistazo a la siguiente función de python que codifica una cadena de texto en una bolsa de palabras. 

In [42]:
def bag_of_words(text):
    """
    
    """
    # global word_encoding
    vocab = {}  # asigna una palabra a un número entero que la representa
    word_encoding = 1
    # crear una lista de todas las palabras en el texto, bien asumir que no hay gramática en nuestro texto para este ejemplo
    words = text.lower().split(" ") 
    # almacena todas las codificaciones y su frecuencia
    bag = {} 

    for word in words:
        if word in vocab:
            encoding = vocab[word]  # obtener la codificación del vocabulario
        else:
            vocab[word]     = word_encoding
            encoding        = word_encoding
            word_encoding  += 1
    
        #if encoding in bag:
        #    bag[encoding] += 1
        #else:
        #    bag[encoding] = 1
        bag[encoding] = bag.get(encoding,0)+1
  
    return bag , vocab

In [43]:
text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)

({1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}, {'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9})


Esta no es realmente la forma en que lo haríamos en la práctica, pero espero que te dé una idea de cómo funciona la bolsa de palabras. Observa que hemos perdido el orden de aparición de las palabras. De hecho, veamos cómo funciona esta codificación para las dos frases que mostramos anteriormente.

In [44]:
def tabla(titulo,bolsa):
    print(titulo+ " :")
    print("+"+"-"*41+"+")
    print(f"|{'palabra':^20}| {'frecuencia':^10} | {'index':5} |")
    print("+"+"-"*41+"+")
    for palabra, index in bolsa[1].items():
        print(f"|{palabra:^20}| {bolsa[0][index]:^10} | {index:>5} |")
    print("+"+"-"*41+"+")

positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

In [45]:
tabla("Positive", pos_bag)

Positive :
+-----------------------------------------+
|      palabra       | frecuencia | index |
+-----------------------------------------+
|         i          |     1      |     1 |
|      thought       |     1      |     2 |
|        the         |     1      |     3 |
|       movie        |     1      |     4 |
|        was         |     2      |     5 |
|       going        |     1      |     6 |
|         to         |     1      |     7 |
|         be         |     1      |     8 |
|        bad         |     1      |     9 |
|        but         |     1      |    10 |
|         it         |     1      |    11 |
|      actually      |     1      |    12 |
|      amazing       |     1      |    13 |
+-----------------------------------------+


In [46]:
tabla("Negative", neg_bag)

Negative :
+-----------------------------------------+
|      palabra       | frecuencia | index |
+-----------------------------------------+
|         i          |     1      |     1 |
|      thought       |     1      |     2 |
|        the         |     1      |     3 |
|       movie        |     1      |     4 |
|        was         |     2      |     5 |
|       going        |     1      |     6 |
|         to         |     1      |     7 |
|         be         |     1      |     8 |
|      amazing       |     1      |     9 |
|        but         |     1      |    10 |
|         it         |     1      |    11 |
|      actually      |     1      |    12 |
|        bad         |     1      |    13 |
+-----------------------------------------+


Podemos ver que, aunque estas frases tienen un significado muy diferente, están codificadas exactamente de la misma manera. Obviamente, esto no va a funcionar. Veamos otros métodos.

### Codificación de enteros
La siguiente técnica que veremos se llama **codificación de enteros**. Se trata de representar cada palabra o carácter de una frase como un único número entero y mantener el orden de estas palabras. Esto debería solucionar el problema que vimos antes de perder el orden de las palabras.


In [63]:
def one_hot_encoding(text):
    vocab = {}  
    word_encoding = 1

    words = text.lower().split(" ") 
    encoding = [] 
    print("+"+"-"*31+"+")
    print(f"|{'Palabra':^20} | {'Codigo':^8}|")
    print("+"+"-"*31+"+")
    for word in words:
        if word in vocab:
            code = vocab[word]  
            encoding.append(code) 
        else:
            vocab[word] = word_encoding
            encoding.append(word_encoding)
            code = vocab[word] 
            word_encoding += 1
            
        print(f"|{word:^20} | {code:^8}|")
    print("+"+"-"*31+"+")
    return encoding , vocab

In [67]:
text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
#print(encoding)
#print(vocab)

+-------------------------------+
|      Palabra        |  Codigo |
+-------------------------------+
|        this         |    1    |
|         is          |    2    |
|         a           |    3    |
|        test         |    4    |
|         to          |    5    |
|        see          |    6    |
|         if          |    7    |
|        this         |    1    |
|        test         |    4    |
|        will         |    8    |
|        work         |    9    |
|         is          |    2    |
|         is          |    2    |
|        test         |    4    |
|         a           |    3    |
|         a           |    3    |
+-------------------------------+


In [68]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

print("Positive:")
pos_encode = one_hot_encoding(positive_review)
print("Negative:")
neg_encode = one_hot_encoding(negative_review)


Positive:
+-------------------------------+
|      Palabra        |  Codigo |
+-------------------------------+
|         i           |    1    |
|      thought        |    2    |
|        the          |    3    |
|       movie         |    4    |
|        was          |    5    |
|       going         |    6    |
|         to          |    7    |
|         be          |    8    |
|        bad          |    9    |
|        but          |    10   |
|         it          |    11   |
|        was          |    5    |
|      actually       |    12   |
|      amazing        |    13   |
+-------------------------------+
Negative:
+-------------------------------+
|      Palabra        |  Codigo |
+-------------------------------+
|         i           |    1    |
|      thought        |    2    |
|        the          |    3    |
|       movie         |    4    |
|        was          |    5    |
|       going         |    6    |
|         to          |    7    |
|         be          |    8

Mucho mejor, ahora llevamos la cuenta del orden de las palabras y podemos saber dónde ocurre cada una. Pero esto todavía tiene algunos problemas. Lo ideal sería que, al codificar las palabras, las palabras similares tuvieran etiquetas similares y las palabras diferentes tuvieran etiquetas muy diferentes. Por ejemplo, las palabras feliz y alegre deberían tener etiquetas muy parecidas para que podamos determinar que son similares. Mientras que palabras como horrible y asombroso deberían tener etiquetas muy diferentes. El método que hemos visto anteriormente no podrá hacer algo así por nosotros. Esto podría significar que el modelo tendrá un tiempo muy difícil para determinar si dos palabras son similares o no, lo que podría resultar en algunos impactos de rendimiento bastante drásticos.

### Word Embeddings
Por suerte, existe un tercer método que es muy superior, la **incrustación de palabras**. Este método mantiene intacto el orden de las palabras y codifica palabras similares con etiquetas muy parecidas. Intenta no sólo codificar la frecuencia y el orden de las palabras, sino también el significado de esas palabras en la frase. Codifica cada palabra como un vector denso que representa su contexto en la frase.

A diferencia de las técnicas anteriores, las incrustaciones de palabras se aprenden observando muchos ejemplos de entrenamiento diferentes. Puede añadir lo que se llama una *capa de incrustación* al principio de su modelo y, mientras éste se entrena, su capa de incrustación aprenderá las incrustaciones correctas de las palabras. También puedes utilizar capas de incrustación preentrenadas.

Esta es la técnica que utilizaremos para nuestros ejemplos y su implementación se mostrará más adelante.

## Redes neuronales recurrentes (RNN)
Ahora que hemos aprendido un poco sobre cómo podemos codificar el texto es el momento de sumergirnos en las redes neuronales recurrentes. Hasta este punto hemos estado utilizando algo llamado **redes neuronales de avance**. Esto significa simplemente que todos nuestros datos se alimentan hacia adelante (todos a la vez) de izquierda a derecha a través de la red. Esto estuvo bien para los problemas que consideramos antes, pero no funcionará muy bien para procesar texto. Después de todo, ni siquiera nosotros (los humanos) procesamos el texto de una sola vez. Leemos palabra por palabra, de izquierda a derecha, y mantenemos un registro del significado actual de la frase para poder entender el significado de la siguiente palabra. Esto es exactamente lo que hace una red neuronal recurrente. Cuando decimos red neuronal recurrente lo que realmente queremos decir es una red que contiene un bucle. Una RNN procesa una palabra a la vez mientras mantiene una memoria interna de lo que ya ha visto. Esto le permitirá tratar las palabras de forma diferente en función de su orden en una frase y construir lentamente una comprensión de toda la entrada, una palabra cada vez.

Por eso tratamos nuestros datos de texto como una secuencia. Para que podamos pasar una palabra a la vez a la RNN.

Veamos cómo podría ser una capa recurrente.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)
*Fuente: https://colah.github.io/posts/2015-08-Understanding-LSTMs/*


Definamos qué significan todas estas variables antes de entrar en la explicación.

**h<sub>t</sub>** Salida en un paso de tiempo t

**x<sub>t</sub>** Entrada  en un paso de tiempo t

**A** Capa Recurrente (bucle)

Lo que su diagrama trata de ilustrar es que una capa recurrente procesa las palabras o la entrada de una en una en combinación con la salida de la iteración anterior. Así, a medida que avanzamos en la secuencia de entrada, construimos una comprensión más compleja del texto en su conjunto.

Lo que acabamos de ver se llama **capa RNN simple**. Puede ser eficaz en el procesamiento de secuencias de texto más cortas para problemas sencillos, pero tiene muchas desventajas asociadas. Uno de ellos es el hecho de que, a medida que las secuencias de texto se hacen más largas, es cada vez más difícil para la red entender el texto correctamente.

## LSTM
La capa que hemos analizado en profundidad anteriormente se llama *RNN simple*. Sin embargo, existen otras capas recurrentes (capas que contienen un bucle) que funcionan mucho mejor que una capa RNN simple. De la que hablaremos aquí se llama LSTM (Long Short-Term Memory). Esta capa funciona de forma muy similar a la capa RNN simple, pero añade una forma de acceder a las entradas de cualquier paso de tiempo en el pasado. Mientras que en nuestra capa RNN simple las entradas de los pasos de tiempo anteriores desaparecían gradualmente a medida que avanzábamos en la entrada. Con una LSTM tenemos una estructura de datos de memoria a largo plazo que almacena todas las entradas vistas anteriormente, así como cuándo las vimos. Esto nos permite acceder a cualquier valor anterior que queramos en cualquier momento. Esto añade complejidad a nuestra red y le permite descubrir más relaciones útiles entre las entradas y el momento en que aparecen. 

Todas las redes neuronales recurrentes tienen la forma de una cadena de módulos repetidos de red neuronal. En RNN estándar, este módulo repetitivo tendrá una estructura muy simple, como una sola capa de tanh.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png)

Los LSTM también tienen esta estructura similar a una cadena, pero el módulo repetitivo tiene una estructura diferente. En lugar de tener una sola capa de red neuronal, hay cuatro que interactúan de una manera muy especial.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

### Recorrido paso a paso de LSTM

#### Primer Paso

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png)

El primer paso en nuestro LSTM es decidir qué información vamos a desechar del estado de la celda. Esta decisión la toma una capa sigmoidea llamada "capa de puerta de olvido". mira $h_{t - 1}$ y $X_{t} $, y genera un número entre 0 y 1 para cada número en el estado de la celda $C_{t - 1}$. Si $f_{t}$ es 1 representa "guardar esto por completo", mientras que un 0 representa "deshacerse completamente de esto".

#### Segundo Paso

El siguiente paso es decidir qué nueva información vamos a almacenar en el estado de la celda. Esto tiene dos partes. Primero, una capa sigmoidea llamada "capa de puerta de entrada" decide qué valores actualizaremos. A continuación, una capa $\tanh$ crea un vector de nuevos valores candidatos, $\tilde{C}_{t}$, que podría agregarse al estado. En el siguiente paso, combinaremos estos dos para crear una actualización del estado.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png)

#### Tercer Paso

Multiplicamos el estado anterior por $f_{t}$ , olvidando las cosas que decidimos olvidar antes. Luego agregamos $i_{t} ∗ \tilde{C}_{t}$. Estos son los nuevos valores candidatos, escalados por cuánto decidimos actualizar cada valor de estado.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png)

### Cuarto Paso

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png) 

Finalmente, tenemos que decidir qué vamos a generar. Esta salida se basará en nuestro estado de celda, pero será una versión filtrada. Primero, ejecutamos una capa sigmoidea que decide qué partes del estado de la celda vamos a generar. Luego, ponemos el estado de la celda a través de $\tanh$ ( para empujar los valores a estar entre −1 y 1 ) y multiplíquelo por la salida de la puerta sigmoidea, de modo que solo emitamos las partes que decidimos.

### Variantes de la memoria a largo plazo
Lo que he descrito hasta ahora es un LSTM bastante normal. Pero no todos los LSTM son iguales a los anteriores. De hecho, parece que casi todos los documentos que involucran LSTM usan una versión ligeramente diferente. Las diferencias son menores, pero vale la pena mencionar algunas de ellas.

Una variante popular de LSTM, presentada por Gers & Schmidhuber , agrega "conexiones de mirilla". Esto significa que dejamos que las capas de la puerta miren el estado de la celda.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-peepholes.png)

Otra variación es utilizar puertas de entrada y de olvido acopladas. En lugar de decidir por separado qué olvidar y qué debemos agregar nueva información, tomamos esas decisiones juntos. Solo olvidamos cuando vamos a ingresar algo en su lugar. Solo ingresamos nuevos valores al estado cuando olvidamos algo más antiguo.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-tied.png)

Una variación un poco más dramática del LSTM es la Unidad Recurrente Cerrada, o GRU, presentada por Cho, et al. (2014) . Combina las puertas de entrada y de olvido en una sola "puerta de actualización". También fusiona el estado de la celda y el estado oculto, y realiza algunos otros cambios. El modelo resultante es más simple que los modelos LSTM estándar y se ha vuelto cada vez más popular.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-GRU.png)


## Análisis del sentimiento
Y ahora es el momento de ver una red neuronal recurrente en acción. Para este ejemplo, vamos a hacer algo llamado análisis de sentimiento.

La definición formal de este término de Wikipedia es la siguiente:

*el proceso de identificar y categorizar computacionalmente las opiniones expresadas en un texto, especialmente para determinar si la actitud del escritor hacia un tema particular, producto, etc. es positiva, negativa o neutral.*

El ejemplo que utilizaremos aquí es la clasificación de las críticas de cine como positivas, negativas o neutras.

*Esta guía está basada en el siguiente tutorial de tensorflow: https://www.tensorflow.org/tutorials/text/text_classification_rnn*

### Conjunto de datos de críticas de películas
Empezamos cargando el conjunto de datos de críticas de películas de IMDB de keras. Este conjunto de datos contiene 25.000 críticas de IMDB donde cada una ya está preprocesada y tiene una etiqueta como positiva o negativa. Cada crítica está codificada por enteros que representan lo común que es una palabra en todo el conjunto de datos. Por ejemplo, una palabra codificada con el número entero 3 significa que es la tercera palabra más común en el conjunto de datos.

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584
MAXLEN     = 250
BATCH_SIZE = 64

In [2]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

17473536/17464789 [==============================] - 2s 0us/step


In [5]:
# Veamos una revisión
print(train_data[1])

[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 4369, 5012, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 8163, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 6853, 5, 163, 11, 3215, 10156, 4, 1153, 9, 194, 775, 7, 8255, 11596, 349, 2637, 148, 605, 15358, 8003, 15, 123, 125, 68, 23141, 6853, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 36893, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 8255, 5, 25249, 656, 245, 2350, 5, 4, 9837, 131, 152, 491, 18, 46151, 32, 7464, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]


### Más preprocesamiento
Si echamos un vistazo a algunas de nuestras revisiones cargadas, nos daremos cuenta de que tienen diferentes longitudes. Esto es un problema. No podemos pasar datos de diferente longitud a nuestra red neuronal. Por lo tanto, debemos hacer que cada reseña tenga la misma longitud. Para ello seguiremos el siguiente procedimiento
- si la reseña tiene más de 250 palabras, se recortan las palabras sobrantes
- si la reseña tiene menos de 250 palabras, añadir la cantidad necesaria de 0's para que sea igual a 250.

Por suerte para nosotros keras tiene una función que puede hacer esto por nosotros:


In [7]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data  = sequence.pad_sequences(test_data,  MAXLEN)

### Creación del modelo
Ahora es el momento de crear el modelo. Utilizaremos una capa de incrustación de palabras como primera capa de nuestro modelo y añadiremos después una capa LSTM que se alimenta de un nodo denso para obtener nuestro sentimiento predicho. 

32 representa la dimensión de salida de los vectores generados por la capa de incrustación. Podemos cambiar este valor si queremos.

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          2834688   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


### Entrenamiento
Ahora es el momento de compilar y entrenar el modelo. 

In [10]:
model.compile( loss      = "binary_crossentropy",
               optimizer = "rmsprop",
               metrics   = ['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 35s 52ms/step - loss: 0.4373 - acc: 0.7983 - val_loss: 0.2946 - val_acc: 0.8846
Epoch 2/10
625/625 [==============================] - 33s 53ms/step - loss: 0.2401 - acc: 0.9090 - val_loss: 0.3154 - val_acc: 0.8610
Epoch 3/10
625/625 [==============================] - 34s 55ms/step - loss: 0.1837 - acc: 0.9337 - val_loss: 0.2854 - val_acc: 0.8796
Epoch 4/10
625/625 [==============================] - 33s 52ms/step - loss: 0.1520 - acc: 0.9452 - val_loss: 0.5194 - val_acc: 0.8248
Epoch 5/10
625/625 [==============================] - 32s 51ms/step - loss: 0.1263 - acc: 0.9548 - val_loss: 0.3043 - val_acc: 0.8902
Epoch 6/10
625/625 [==============================] - 35s 56ms/step - loss: 0.1100 - acc: 0.9632 - val_loss: 0.3056 - val_acc: 0.8884
Epoch 7/10
625/625 [==============================] - 31s 50ms/step - loss: 0.0934 - acc: 0.9685 - val_loss: 0.3248 - val_acc: 0.8812
Epoch 8/10
625/625 [==============================] - 34s 54ms

Y evaluaremos el modelo en nuestros datos de entrenamiento para ver su rendimiento.

In [11]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 11s 13ms/step - loss: 0.4405 - acc: 0.8433
[0.4405488669872284, 0.8433200120925903]


### Haciendo predicciones
Ahora vamos a utilizar nuestra red para hacer predicciones sobre nuestras propias reseñas. 

Como nuestras opiniones están codificadas, tenemos que convertir cualquier opinión que escribamos en ese formato para que la red pueda entenderla. Para ello, cargamos las codificaciones del conjunto de datos y las utilizamos para codificar nuestros propios datos.

In [12]:
word_index = imdb.get_word_index()

def encode_text(text):
    tokens = keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1654784/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [14]:
# vamos a hacer una función de decodificación

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
        if num != PAD:
            text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [15]:
def predict(text):
    encoded_text = encode_text(text)
    pred         = np.zeros((1,250))
    pred[0]      = encoded_text
    result       = model.predict(pred) 
    print(result[0])

In [16]:
positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

[0.7291384]


In [17]:
negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.20502672]


## Generador de juegos RNN

Ahora es el momento de uno de los ejemplos más geniales que hemos visto hasta ahora. Vamos a utilizar una RNN para generar una obra de teatro. Simplemente mostraremos a la RNN un ejemplo de algo que queremos que recree y ella aprenderá a escribir una versión de la misma por sí misma. Haremos esto utilizando un modelo de predicción de caracteres que tomará como entrada una secuencia de longitud variable y predecirá el siguiente carácter. Podemos usar el modelo muchas veces seguidas con la salida de la última predicción como entrada para la siguiente llamada para generar una secuencia.


*Esta guía se basa en lo siguiente: https://www.tensorflow.org/tutorials/text/text_generation*

### Dataset
Para este ejemplo, sólo necesitamos una pieza de datos de entrenamiento. De hecho, podemos escribir nuestro propio poema u obra de teatro y pasarlo a la red para el entrenamiento si queremos. Sin embargo, para facilitar las cosas, utilizaremos un extracto de una obra de teatro de Shakesphere.

In [18]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


In [ ]:
# para ejucuar en la colab de google
from google.colab import files
path_to_file = list(files.upload().keys())[0]

In [19]:
# Vemos donde guardo el archivo
path_to_file

'/home/emi/.keras/datasets/shakespeare.txt'

In [21]:
# leemos el archivo y mostramops la longitud en caracteres
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('longitud del archivo: {} caracteres'.format(len(text)))

longitud del archivo: 1115394 caracteres


In [22]:
# imprimimos los primeros 250 caracteres
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Encodificación
Como este texto no está codificado todavía, tenemos que hacerlo nosotros mismos. Vamos a codificar cada carácter único como un entero diferente.


In [28]:
vocab = sorted(set(text))
# Creación de un mapeo de caracteres únicos a índices
char2idx = {u:i for i, u in enumerate(vocab)}
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [29]:
idx2char = np.array(vocab)
idx2char

array(['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [30]:
# Creo una funcion lambda que recibe un texto y devuelve un array con los valores en enteros)
text_to_int = lambda text : np.array([char2idx[c] for c in text])

In [40]:
text_as_int = text_to_int(text)

In [43]:
text_as_int

array([18, 47, 56, ..., 45,  8,  0])

In [34]:
# veamos cómo se codifica parte de nuestro texto
print("Texto:", text[:13])
print("Encode:", text_to_int(text[:13]))

Texto: First Citizen
Encode: [18 47 56 57 58  1 15 47 58 47 64 43 52]


Y aquí haremos una función que pueda convertir nuestros valores numéricos en texto.

In [45]:
def int_to_text(ints):
    # probamos si ints es un tensor de tensorflow lo tranformamos a un array de numpy
    try:
        ints = ints.numpy()
    except:
        pass
    return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


### Creación de ejemplos de entrenamiento
Recuerde que nuestra tarea es alimentar al modelo con una secuencia y hacer que nos devuelva el siguiente carácter. Esto significa que tenemos que dividir nuestros datos de texto de arriba en muchas secuencias más cortas que podamos pasar al modelo como ejemplos de entrenamiento. 

Los ejemplos de entrenamiento que prepararemos utilizarán una secuencia *longitud_de_secuencia* como entrada y una secuencia *longitud_de_secuencia* como salida, donde esa secuencia es la secuencia original desplazada una letra a la derecha. Por ejemplo:

``Entrada: Infierno | salida: ello```

Nuestro primer paso será crear una secuencia de caracteres a partir de nuestros datos de texto.

In [46]:
seq_length = 100  # longitud de la secuencia para un ejemplo de entrenamiento
examples_per_epoch = len(text)//(seq_length+1)

# # Crear ejemplos de formación / objetivos
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

A continuación, podemos utilizar el método de lotes para convertir este flujo de caracteres en lotes de la longitud deseada.

In [47]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder = True)

Ahora tenemos que utilizar estas secuencias de longitud 101 y dividirlas en entrada y salida.

In [49]:
def split_input_target(chunk):  # para el primer ejemplo: hello
    input_text  = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # utilizamos map para aplicar la función anterior a cada entrada

In [50]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [54]:
for x, y in dataset.take(1):
    print(x.numpy())
    print("\n\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print(y.numpy())
    print("\nOUTPUT")
    print(int_to_text(y))

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]


EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


Por último, tenemos que hacer lotes de entrenamiento.

In [56]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Tamaño del buffer para barajar el conjunto de datos
# (TF data está diseñado para trabajar con secuencias posiblemente infinitas,
# por lo que no intenta barajar toda la secuencia en memoria. En su lugar,
# mantiene un buffer en el que baraja los elementos).

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [57]:
data

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

### Construir el modelo
Ahora es el momento de construir el modelo. Utilizaremos una capa de incrustación, una LSTM y una capa densa que contiene un nodo para cada carácter único en nuestros datos de entrenamiento. La capa densa nos dará una distribución de probabilidad sobre todos los nodos.

In [58]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size, 
                                      embedding_dim,
                                      batch_input_shape = [batch_size, None]),
            tf.keras.layers.LSTM(rnn_units,
                                 # sreturn_sequences: Booleano. Si se devuelve la última salida. en la secuencia de salida, 
                                 # o la secuencia completa. Por defecto: Falso.
                                 return_sequences      = True,
                                 # stateful: Booleano (por defecto Falso). 
                                 # Si es True, el último estado de cada muestra de índice i en un lote se utilizará 
                                 # como estado inicial para la muestra de índice i en el siguiente lote.
                                 stateful              = True,
                                 # recurrent_initializer: Inicializador para la matriz de pesos de recurrent_kernel, 
                                 # utilizada para la transformación lineal del estado recurrente. Por defecto: ortogonal
                                 recurrent_initializer = 'glorot_uniform'),
            tf.keras.layers.Dense(vocab_size)
                                ])
    return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


## Creación de una función de pérdida
Ahora vamos a crear nuestra propia función de pérdida para este problema. Esto se debe a que nuestro modelo producirá un tensor con forma de (64, longitud_de_secuencia, 65) que representa la distribución de probabilidad de cada carácter en cada paso de tiempo para cada secuencia del lote.   

Sin embargo, antes de hacerlo, echemos un vistazo a un ejemplo de entrada y a la salida de nuestro modelo no entrenado. Esto es para que podamos entender lo que el modelo nos está dando.

In [60]:
for input_example_batch, target_example_batch in data.take(1):
    # pide a nuestro modelo una predicción sobre nuestro primer lote de datos de entrenamiento (64 entradas)
    example_batch_predictions = model(input_example_batch)  
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [61]:
# Podemos ver que la predicción es un array de 64 arrays, uno por cada entrada del lote
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-3.12169059e-03  6.64685853e-04 -1.41554140e-03 ...  1.01210969e-03
    8.10748525e-03  1.59163424e-03]
  [-7.11657712e-03  1.86954718e-03 -3.77903634e-05 ...  3.40952212e-03
    2.41800584e-03  3.94266006e-03]
  [-4.16120281e-03  3.97378253e-03 -2.25786888e-03 ...  6.56198943e-03
    3.02954996e-03  4.16458864e-03]
  ...
  [-6.92722108e-03 -3.01308464e-03  4.83981986e-03 ...  5.85018471e-03
   -3.67472740e-03 -2.18911609e-03]
  [-4.60554799e-03  7.14544265e-04  1.13847316e-03 ...  2.15948443e-03
    1.29446713e-03  1.05620315e-03]
  [ 3.61024926e-04  2.83821439e-03  1.90595165e-05 ...  3.26792034e-03
   -4.66392934e-03  3.79081373e-03]]

 [[ 1.81365060e-04 -8.49255826e-04  1.62916854e-02 ... -6.68749213e-03
   -6.36514416e-03 -1.14551960e-02]
  [ 1.54794916e-03  1.39831216e-03  1.63167082e-02 ... -2.22772593e-03
   -8.82198103e-03 -2.06356961e-03]
  [-3.22013395e-03  7.26652099e-03  1.60768647e-02 ... -1.58064463e-03
   -3.37851094e-03 -5.82813472e-03]
  ...
  [-9.974

In [62]:
# examinemos una predicción
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# observe que esta es una matriz 2d de longitud 100, donde cada matriz interior es la predicción para el siguiente carácter en cada paso de tiempo

100
tf.Tensor(
[[-3.1216906e-03  6.6468585e-04 -1.4155414e-03 ...  1.0121097e-03
   8.1074852e-03  1.5916342e-03]
 [-7.1165771e-03  1.8695472e-03 -3.7790363e-05 ...  3.4095221e-03
   2.4180058e-03  3.9426601e-03]
 [-4.1612028e-03  3.9737825e-03 -2.2578689e-03 ...  6.5619894e-03
   3.0295500e-03  4.1645886e-03]
 ...
 [-6.9272211e-03 -3.0130846e-03  4.8398199e-03 ...  5.8501847e-03
  -3.6747274e-03 -2.1891161e-03]
 [-4.6055480e-03  7.1454427e-04  1.1384732e-03 ...  2.1594844e-03
   1.2944671e-03  1.0562032e-03]
 [ 3.6102493e-04  2.8382144e-03  1.9059516e-05 ...  3.2679203e-03
  -4.6639293e-03  3.7908137e-03]], shape=(100, 65), dtype=float32)


In [63]:
# y finalmente veremos una predicción en el primer paso de tiempo
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# y  por supuesto, sus 65 valores que representan la probabilidad de que cada carácter ocurra a continuación

65
tf.Tensor(
[-0.00312169  0.00066469 -0.00141554 -0.00551986  0.00704249  0.01112877
 -0.00523458  0.00175228  0.00127487 -0.00229272 -0.01128815  0.00097445
 -0.00437749 -0.00355143  0.00077764 -0.00708889 -0.00251785 -0.00164459
  0.00573261 -0.00639464  0.00461333 -0.00145566 -0.0065794   0.00724377
  0.00153553  0.00665463  0.00233206 -0.00682848 -0.00813593  0.01568121
  0.00250032  0.00017275 -0.00796703  0.00607542 -0.00515135 -0.00674238
  0.0146931  -0.00952782 -0.01442668  0.00839407  0.00714136  0.00648044
  0.00127929 -0.00141731 -0.00330456  0.00417646  0.0023957   0.00606689
  0.00856928  0.01402924  0.00154698  0.00868561 -0.00942474 -0.01210114
 -0.01619386  0.00139606  0.01041866  0.01015501  0.00956712  0.00808504
 -0.00159678  0.00201729  0.00101211  0.00810749  0.00159163], shape=(65,), dtype=float32)


In [64]:
# Si queremos determinar el carácter predicho, necesitamos muestrear la distribución de salida (elegir un valor basado en la probabilidad)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# ahora podemos remodelar esa matriz y convertir todos los enteros en números para ver los caracteres reales
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # y esto es lo que predijo el modelo para la secuencia de entrenamiento 1

"-$iMt;KMZRDAJf!ZX OdUPQ.mqotf?UQguxkVtzT-WY$pZEpX.TV.WUTMljTh!qUfrfZoV'\nKw-U:H,NUl-FRSPWaSeAgh:C3.hv"

Así que ahora tenemos que crear una función de pérdida que pueda comparar esa salida con la salida esperada y darnos algún valor numérico que represente lo cerca que estaban las dos. 

In [65]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

### Compilación del modelo
En este punto podemos pensar en nuestro problema como un problema de clasificación en el que el modelo predice la probabilidad de que cada letra única sea la siguiente.

In [66]:
model.compile(optimizer='adam', loss=loss)

### Creación de puntos de control (checkpoints)
Ahora vamos a configurar nuestro modelo para que guarde los puntos de control mientras se entrena. Esto nos permitirá cargar nuestro modelo desde un punto de control y continuar entrenándolo.

In [67]:
# Directorio 
checkpoint_dir = './training_checkpoints'
# Nombre del checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath          = checkpoint_prefix,
                                save_weights_only = True)

### Entrenamiento
Por último, empezaremos a entrenar el modelo. 
Puede correr en una colab con GPU  
**Si esto tarda un poco vaya a Runtime > Change Runtime Type y elija "GPU" en acelerador de hardware.**


In [68]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 687s 4s/step - loss: 2.6396
Epoch 2/50
172/172 [==============================] - 643s 4s/step - loss: 1.9299
Epoch 3/50
172/172 [==============================] - 610s 4s/step - loss: 1.6740
Epoch 4/50
172/172 [==============================] - 615s 4s/step - loss: 1.5307
Epoch 5/50
172/172 [==============================] - 623s 4s/step - loss: 1.4443
Epoch 6/50
172/172 [==============================] - 609s 4s/step - loss: 1.3868
Epoch 7/50
172/172 [==============================] - 606s 4s/step - loss: 1.3417
Epoch 8/50
172/172 [==============================] - 598s 3s/step - loss: 1.3037
Epoch 9/50
172/172 [==============================] - 602s 3s/step - loss: 1.2685
Epoch 10/50
172/172 [==============================] - 603s 3s/step - loss: 1.2353
Epoch 11/50
172/172 [==============================] - 626s 4s/step - loss: 1.2026
Epoch 12/50
172/172 [==============================] - 621s 4s/step - loss: 1.1695
Epoch 13/50
1

### Loading the Model
Reconstruiremos el modelo a partir de un punto de control utilizando un batch_size de 1 para que podamos alimentar un trozo de texto al modelo y que éste haga una predicción.

In [69]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Una vez que el modelo ha terminado de entrenar, podemos encontrar el **último punto de control** que almacena los pesos del modelo utilizando la siguiente línea.

In [70]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

### Generación de texto
Ahora podemos utilizar la encantadora función proporcionada por tensorflow para generar algo de texto utilizando cualquier cadena de inicio que queramos.

In [71]:
def generate_text(model, start_string):
    # Paso de evaluación (generar texto usando el modelo aprendido)
    # Número de caracteres a generar
    num_generate = 800

    # Convertir nuestra cadena de inicio en números (vectorización)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # lista vacia para texto generado
    text_generated = []

    # Las temperaturas bajas dan como resultado un texto más predecible.
    # Las temperaturas más altas dan como resultado un texto más sorprendente.
    # Experimente para encontrar la mejor configuración.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # eliminar la dimensión del lote
    
        predictions = tf.squeeze(predictions, 0)

        # utilizando una distribución categórica para predecir el carácter devuelto por el modelo
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pasamos el carácter predicho como la siguiente entrada al modelo
        # junto con el estado oculto anterior
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [72]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string:  be or not to be


be or not to be done:
Shall, what most I, being over-proud intentle proud;
For by the English personall pluck
Ejoubted our law cameling; though we would wish
Your country's brown, a ring form on's blood, be it thankil,
That young Prince Edward marries Warwick's daughter.

KING EDWARD IV:
An oack, to say to one immaker: shall I send
Dis this for ill scope be wakent good awhile! would I,
Take thou on every time what should not
beat you that act of it?

Second Capulet:
'Sicil the other, there it is berefactors
Are crack'd for't: both you, as I hear, must I be glass,
I then cast out again and that sayst do in piench my mother,
How doth the portion and suffer looking sla, whereof thy love, or or?
I might commend me to thy living lior
Is as a man divine and myself,
And not against his eased before all hearts b


## Fuentes

1. “Text Classification with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_classification_rnn.
2. “Text Generation with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_generation.
3. “Understanding LSTM Networks.” Understanding LSTM Networks -- Colah's Blog, https://colah.github.io/posts/2015-08-Understanding-LSTMs/.
4. Chollet François. Deep Learning with Python. Manning Publications Co., 2018.